In [ ]:
# Force compatible package versions for Colab + TF 2.19
# - numpy pinned to <2.2
# - pandas pinned to 2.2.2 (matches google-colab requirement)
# - scipy pinned to a version compatible with numpy
!pip -q install "numpy==2.0.2" "pandas==2.2.2" "matplotlib==3.9.0" "scipy==1.13.1"

# IMPORTANT: Restart runtime so the pinned versions are actually used.
import os; os.kill(os.getpid(), 9)

# CNN Task Experiments (Colab-ready)

This notebook runs the experiments requested in Week 10 using a simple CNN on CIFAR-10:

- Kernels 3×3 with epochs 10, 20, 50
- Kernels 5×5 with epochs 10, 20

It produces:
- Accuracy plots (PNG) for each run
- Summary CSV and JSON files for each kernel size

How to use:
1. Runtime → Change runtime type → Select GPU (recommended)
2. Run the cells from top to bottom
3. Download the generated files from /content/results (or zip them in the last cell)
4. Place them into week10/answers and tell me; I will append the exact numbers to answers.txt


In [ ]:
# Keep Colab’s TF-compatible versions (avoid upgrading numpy/pandas)
!pip -q install --upgrade --no-deps "matplotlib==3.9.0"

import os, time, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, models, layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

print('TensorFlow:', tf.__version__)
print('GPU available?', tf.config.list_physical_devices('GPU'))


In [ ]:
def build_model(kernel_size=(3, 3)):
    model = models.Sequential()
    model.add(layers.Conv2D(32, kernel_size, activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, kernel_size, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, kernel_size, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))  # logits
    model.compile(optimizer=Adam(),
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

def load_data():
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
    train_images = train_images.astype('float32') / 255.0
    test_images  = test_images.astype('float32') / 255.0
    train_labels = train_labels.reshape(-1)
    test_labels  = test_labels.reshape(-1)
    return (train_images, train_labels), (test_images, test_labels)

os.makedirs('/content/results', exist_ok=True)


In [ ]:
def run_experiment(epochs, kernel_size, out_dir):
    (train_images, train_labels), (test_images, test_labels) = load_data()
    model = build_model(kernel_size=kernel_size)
    start = time.time()
    history = model.fit(
        train_images, train_labels,
        epochs=epochs,
        batch_size=128,
        validation_data=(test_images, test_labels),
        verbose=1
    )
    duration = time.time() - start
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)

    # Accuracy learning curve
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['accuracy'], label='train_accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'Accuracy (epochs={epochs}, kernel={kernel_size})\nTest acc={test_acc:.4f}, time={duration/60:.1f} min')
    plt.legend()
    acc_plot_path = os.path.join(out_dir, f"cnn_task_acc_epochs_{epochs}_kernel_{kernel_size[0]}x{kernel_size[1]}.png")
    plt.tight_layout()
    plt.savefig(acc_plot_path)
    plt.close()

    # Loss learning curve
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Loss (epochs={epochs}, kernel={kernel_size})')
    plt.legend()
    loss_plot_path = os.path.join(out_dir, f"cnn_task_loss_epochs_{epochs}_kernel_{kernel_size[0]}x{kernel_size[1]}.png")
    plt.tight_layout()
    plt.savefig(loss_plot_path)
    plt.close()

    # Confusion matrix on test set
    logits = model.predict(test_images, batch_size=256, verbose=0)
    y_pred = np.argmax(logits, axis=1)
    cm = tf.math.confusion_matrix(test_labels, y_pred, num_classes=10).numpy()

    plt.figure(figsize=(7, 6))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.title(f'Confusion Matrix (epochs={epochs}, kernel={kernel_size})')
    plt.colorbar()
    tick_marks = np.arange(10)
    plt.xticks(tick_marks, tick_marks)
    plt.yticks(tick_marks, tick_marks)
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     ha='center', va='center',
                     color='white' if cm[i, j] > thresh else 'black')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    cm_plot_path = os.path.join(out_dir, f"cnn_task_cm_epochs_{epochs}_kernel_{kernel_size[0]}x{kernel_size[1]}.png")
    plt.savefig(cm_plot_path)
    plt.close()

    return {
        'epochs': int(epochs),
        'kernel_size': f"{kernel_size[0]}x{kernel_size[1]}",
        'test_acc': float(test_acc),
        'test_loss': float(test_loss),
        'duration_sec': float(duration),
        'acc_plot': os.path.basename(acc_plot_path),
        'loss_plot': os.path.basename(loss_plot_path),
        'cm_plot': os.path.basename(cm_plot_path),
    }


def run_battery(epochs_list, kernel_size, out_dir='/content/results'):
    results = []
    for e in epochs_list:
        results.append(run_experiment(e, kernel_size, out_dir))
    df = pd.DataFrame(results)
    csv_path = os.path.join(out_dir, f"cnn_task_results_kernel_{kernel_size[0]}x{kernel_size[1]}.csv")
    json_path = os.path.join(out_dir, f"cnn_task_results_kernel_{kernel_size[0]}x{kernel_size[1]}.json")
    df.to_csv(csv_path, index=False)
    with open(json_path, 'w') as f:
        json.dump(results, f, indent=2)
    return df, csv_path, json_path


Tip: If training is slow on CPU, go to Runtime → Change runtime type → Hardware accelerator: GPU, then run the first cell (version pin) and restart automatically, then run all other cells.

In [ ]:
# 3x3 kernels: 10, 20, 50 epochs
df_3x3, csv_3x3, json_3x3 = run_battery([10, 20, 50], (3, 3), out_dir='/content/results')
print('3x3 results:')
display(df_3x3)
print('Saved:', csv_3x3, json_3x3)

# 5x5 kernels: 10, 20 epochs
df_5x5, csv_5x5, json_5x5 = run_battery([10, 20], (5, 5), out_dir='/content/results')
print('5x5 results:')
display(df_5x5)
print('Saved:', csv_5x5, json_5x5)


In [ ]:
# Zip all results for easy download
import shutil
shutil.make_archive('/content/cnn_task_results', 'zip', '/content/results')
print('Zipped -> /content/cnn_task_results.zip')
